In [1]:
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
import numpy as np
from sklearn.metrics import classification_report, mean_squared_error, mean_absolute_error


In [2]:
# 1) Cargar dataset
(X_train, y_train), (X_test, y_test) = boston_housing.load_data()


In [3]:
import pandas as pd

feature_names = [
    "CRIM",      # Per capita crime rate by town
    "ZN",        # Proportion of residential land zoned for lots > 25k sq.ft
    "INDUS",     # Proportion of non-retail business acres per town
    "CHAS",      # Charles River dummy variable (= 1 if tract bounds river)
    "NOX",       # Nitric oxides concentration (parts per 10 million)
    "RM",        # Average number of rooms per dwelling
    "AGE",       # Proportion of owner-occupied units built prior to 1940
    "DIS",       # Weighted distances to five Boston employment centers
    "RAD",       # Index of accessibility to radial highways
    "TAX",       # Full-value property tax rate per $10,000
    "PTRATIO",   # Pupil-teacher ratio by town
    "B",         # 1000(Bk - 0.63)^2 where Bk is % of Black residents
    "LSTAT"      # % lower status of the population
]
# Convertir a dataframe
df_train = pd.DataFrame(X_train, columns=feature_names)

# Inspeccionar tipos y primeros valores
print(df_train.info())
print(df_train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404 entries, 0 to 403
Data columns (total 13 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   CRIM     404 non-null    float64
 1   ZN       404 non-null    float64
 2   INDUS    404 non-null    float64
 3   CHAS     404 non-null    float64
 4   NOX      404 non-null    float64
 5   RM       404 non-null    float64
 6   AGE      404 non-null    float64
 7   DIS      404 non-null    float64
 8   RAD      404 non-null    float64
 9   TAX      404 non-null    float64
 10  PTRATIO  404 non-null    float64
 11  B        404 non-null    float64
 12  LSTAT    404 non-null    float64
dtypes: float64(13)
memory usage: 41.2 KB
None
      CRIM    ZN  INDUS  CHAS    NOX     RM    AGE     DIS   RAD    TAX  \
0  1.23247   0.0   8.14   0.0  0.538  6.142   91.7  3.9769   4.0  307.0   
1  0.02177  82.5   2.03   0.0  0.415  7.610   15.7  6.2700   2.0  348.0   
2  4.89822   0.0  18.10   0.0  0.631  4.970  100.0  1

In [4]:
# 2) Crear la salida binaria (alto/bajo según mediana)
median = np.median(y_train)
y_train_bin = (y_train > median).astype(int)
y_test_bin  = (y_test  > median).astype(int)


In [5]:
# 3) Construir red multitarea
inp = Input(shape=(13,))
h = Dense(32, activation='relu')(inp)
h = Dense(16, activation='relu')(h)

reg = Dense(1, name='reg')(h)                        # Salida regresión
clas = Dense(1, activation='sigmoid', name='clas')(h) # Salida clasificación

modelo = Model(inp, [reg, clas])

modelo.compile(
    optimizer='adam',
    loss={'reg': 'mse', 'clas': 'binary_crossentropy'},
    metrics={'reg': 'mae', 'clas': 'accuracy'}
)

modelo.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 13)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 32)           448         ['input_1[0][0]']                
                                                                                                  
 dense_1 (Dense)                (None, 16)           528         ['dense[0][0]']                  
                                                                                                  
 reg (Dense)                    (None, 1)            17          ['dense_1[0][0]']                
                                                                                              

In [6]:
# 4) Entrenamiento
hist = modelo.fit(
    X_train,
    {'reg': y_train, 'clas': y_train_bin},     # dos targets
    epochs=50,
    batch_size=16,
    validation_split=0.2,
    verbose=1
)

Epoch 1/50
21/21 [==============================] - 2s 23ms/step - loss: 655.7843 - reg_loss: 587.7502 - clas_loss: 68.0341 - reg_mae: 18.0253 - clas_accuracy: 0.4892 - val_loss: 235.6803 - val_reg_loss: 183.0252 - val_clas_loss: 52.6551 - val_reg_mae: 12.5067 - val_clas_accuracy: 0.5432
Epoch 2/50
21/21 [==============================] - 0s 3ms/step - loss: 154.8239 - reg_loss: 103.2825 - clas_loss: 51.5413 - reg_mae: 7.7269 - clas_accuracy: 0.4830 - val_loss: 125.7215 - val_reg_loss: 82.7589 - val_clas_loss: 42.9626 - val_reg_mae: 6.3264 - val_clas_accuracy: 0.5432
Epoch 3/50
21/21 [==============================] - 0s 3ms/step - loss: 110.7591 - reg_loss: 70.2172 - clas_loss: 40.5419 - reg_mae: 6.3979 - clas_accuracy: 0.4799 - val_loss: 110.4462 - val_reg_loss: 77.9808 - val_clas_loss: 32.4654 - val_reg_mae: 6.0018 - val_clas_accuracy: 0.5185
Epoch 4/50
21/21 [==============================] - 0s 3ms/step - loss: 93.5119 - reg_loss: 63.5981 - clas_loss: 29.9139 - reg_mae: 5.6965 - c

21/21 [==============================] - 0s 3ms/step - loss: 43.4744 - reg_loss: 42.1713 - clas_loss: 1.3031 - reg_mae: 4.5871 - clas_accuracy: 0.6378 - val_loss: 54.5166 - val_reg_loss: 53.1446 - val_clas_loss: 1.3720 - val_reg_mae: 4.9919 - val_clas_accuracy: 0.5926
Epoch 31/50
21/21 [==============================] - 0s 3ms/step - loss: 42.3013 - reg_loss: 41.0056 - clas_loss: 1.2957 - reg_mae: 4.4466 - clas_accuracy: 0.6780 - val_loss: 54.2492 - val_reg_loss: 52.9850 - val_clas_loss: 1.2643 - val_reg_mae: 5.0341 - val_clas_accuracy: 0.6790
Epoch 32/50
21/21 [==============================] - 0s 5ms/step - loss: 42.6746 - reg_loss: 41.4540 - clas_loss: 1.2207 - reg_mae: 4.4476 - clas_accuracy: 0.6935 - val_loss: 51.1836 - val_reg_loss: 49.9233 - val_clas_loss: 1.2603 - val_reg_mae: 5.3454 - val_clas_accuracy: 0.7037
Epoch 33/50
21/21 [==============================] - 0s 4ms/step - loss: 41.8223 - reg_loss: 40.5301 - clas_loss: 1.2922 - reg_mae: 4.5016 - clas_accuracy: 0.6254 - val_

In [7]:
# 5) Evaluación en test set
eval_results = modelo.evaluate(
    X_test,
    {'reg': y_test, 'clas': y_test_bin},
    verbose=0
)

print("\n=== Evaluación en test ===")
print(f"Loss total:                  {eval_results[0]:.4f}")
print(f"Loss regresión (reg):        {eval_results[1]:.4f}")
print(f"MAE regresión (reg):         {eval_results[2]:.4f}")
print(f"Loss clasificación (clas):   {eval_results[3]:.4f}")
print(f"Accuracy clasificación:       {eval_results[4]:.4f}")



=== Evaluación en test ===
Loss total:                  45.8349
Loss regresión (reg):        44.1227
MAE regresión (reg):         1.7121
Loss clasificación (clas):   5.0446
Accuracy clasificación:       0.6667


In [8]:
# 6) Predicciones
y_pred_reg, y_pred_clas_prob = modelo.predict(X_test)

# Para clasificación: pasar de probabilidad a clase
y_pred_clas = (y_pred_clas_prob > 0.5).astype(int)

# 7) Métricas detalladas

print("\n=== Métricas regresión ===")
print(f"MSE:  {mean_squared_error(y_test, y_pred_reg):.3f}")
print(f"MAE:  {mean_absolute_error(y_test, y_pred_reg):.3f}")

print("\n=== Reporte de clasificación ===")
print(classification_report(y_test_bin, y_pred_clas))


4/4 [==============================] - 0s 2ms/step

=== Métricas regresión ===
MSE:  44.123
MAE:  5.045

=== Reporte de clasificación ===
              precision    recall  f1-score   support

           0       0.54      0.85      0.66        39
           1       0.85      0.56      0.67        63

    accuracy                           0.67       102
   macro avg       0.70      0.70      0.67       102
weighted avg       0.73      0.67      0.67       102

